# MNIST on SageMaker with PyTorch Lightning

In [22]:
# ! pip install pytorch_lightning
# ! pip install ipywidgets
# ! jupyter nbextension enable --py widgetsnbextension
# ! pip install scikit-image

In [24]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl

import os
from skimage import io, transform

In [25]:
import matplotlib.pyplot as plt

## Download and prepare data

In [14]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.conv_layer_1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,28, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2))
        self.conv_layer_2 = torch.nn.Sequential(
        torch.nn.Conv2d(28,10, kernel_size=2),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2))
        self.dropout1=torch.nn.Dropout(0.25)
        self.fully_connected_1=torch.nn.Linear(250,18)
        self.dropout2=torch.nn.Dropout(0.08)
        self.fully_connected_2=torch.nn.Linear(18,10)

    def prepare_data(self):
        MNIST(os.getcwd(), train=True, download=True)
        MNIST(os.getcwd(), train=False, download=True)
        mnist_images = MNIST(os.getcwd(), train=True, download=False, transform=transforms.ToTensor())
        
        # Splitting into validation and test set
        self.train_set, self.val_set = random_split(mnist_images,[55000, 5000])

        
    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=128, num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=128, num_workers=4)
    
#    def test_dataloader(self):
#        return DataLoader(MNIST(os.getcwd(), train=False, download=False, transform=transform.ToTensor()), batch_size=128)
    
    def forward(self,x):
        x=self.conv_layer_1(x)
        x=self.conv_layer_2(x)
        x=self.dropout1(x)
        x=torch.relu(self.fully_connected_1(x.view(x.size(0),-1)))
        x=F.leaky_relu(self.dropout2(x))
        return F.softmax(self.fully_connected_2(x), dim=1)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    
    def training_step(self, batch, batch_idx):
        
        # Get input and output from batch
        x, labels = batch
        
        # Compute prediction through the network
        prediction = self.forward(x)
        
        loss = F.nll_loss(prediction, labels)
        
        # Logs training loss
        logs={'train_loss':loss}
        
        output = {
            # This is required in training to be used by backpropagation
            'loss':loss,
            # This is optional for logging pourposes
            'log':logs
        }
        
        return output
    
    def validation_step(self, batch, batch_idx):
        x, labels = batch
        prediction = self.forward(x)
        return {
            'val_loss': torch.nn.functional.cross_entropy(prediction, labels)
        }
    
    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        logs = {'val_loss':avg_loss}
        return {
            'avg_val_loss':avg_loss,
            'log':logs
        }

In [15]:
# The trainer abstracts training, validation and test loops

mnistTrainer=pl.Trainer(gpus=1, max_epochs=100)

model = MNISTClassifier()
mnistTrainer.fit(model)

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name              | Type       | Params
-------------------------------------------------
0 | conv_layer_1      | Sequential | 728   
1 | conv_layer_2      | Sequential | 1 K   
2 | dropout1          | Dropout    | 0     
3 | fully_connected_1 | Linear     | 4 K   
4 | dropout2          | Dropout    | 0     
5 | fully_connected_2 | Linear     | 190   
INFO:lightning:
  | Name              | Type       | Params
------------------------------------------------

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


1

In [37]:
! wget https://github.com/myleott/mnist_png/raw/master/mnist_png.tar.gz

--2020-07-11 21:12:08--  https://github.com/myleott/mnist_png/raw/master/mnist_png.tar.gz
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/myleott/mnist_png/master/mnist_png.tar.gz [following]
--2020-07-11 21:12:08--  https://raw.githubusercontent.com/myleott/mnist_png/master/mnist_png.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.24.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.24.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15683414 (15M) [application/octet-stream]
Saving to: 'mnist_png.tar.gz'

mnist_png.tar.gz    100%[===================>]  14.96M  --.-KB/s    in 0.1s    

2020-07-11 21:12:10 (149 MB/s) - 'mnist_png.tar.gz' saved [15683414/15683414]



In [ ]:
#! cd mnist_dataset && tar xvf mnist_png.tar.gz
#! ls -la mnist_dataset
#! chmod 777 -R mnist_dataset/mnist_png
! cd mnist_dataset &&  aws s3 sync s3://dataset.mnist .

In [ ]:
c